Let's work through a Text2SQL use case where we are starting from scratch without a nice and clean dataset of questions, SQL queries, or expected responses.

In [ ]:
!pip install openai duckdb datasets pyarrow pydantic nest_asyncio --quiet

Let's first start a phoenix server. Note that this is not necessary if you have a phoenix server running already.

In [ ]:
import phoenix as px

px.launch_app()

Let's also setup tracing for OpenAI as we will be using their API to perform the synthesis.

In [ ]:
from phoenix.trace.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument()

Let's make sure we can run async code in the notebook.

In [ ]:
import nest_asyncio

nest_asyncio.apply()

Lastly, let's make sure we have our openai API key set up.

In [ ]:
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

## Download Data

We are going to use the NBA dataset that information from 2014 - 2018. We will use DuckDB as our database.

In [ ]:
import duckdb
from datasets import load_dataset

data = load_dataset("suzyanil/nba-data")["train"]

conn = duckdb.connect(database=":memory:", read_only=False)
conn.register("nba", data.to_pandas())

conn.query("SELECT * FROM nba LIMIT 5").to_df().to_dict(orient="records")[0]

## Implement Text2SQL

Let's start by implementing a simple text2sql logic.

In [ ]:
import os
from textwrap import dedent

import openai

client = openai.AsyncClient()

columns = conn.query("DESCRIBE nba").to_df().to_dict(orient="records")

TASK_MODEL = "gpt-4o"

columns_str = ",".join(column["column_name"] + ": " + column["column_type"] for column in columns)
system_prompt = dedent(f"""
You are a SQL expert, and you are given a single table named nba with the following columns:
{columns_str}

Write a SQL query corresponding to the user's
request. Return just the query text, with no formatting (backticks, markdown, etc.).""")


async def generate_query(input):
    response = await client.chat.completions.create(
        model=TASK_MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return response.choices[0].message.content

In [ ]:
query = await generate_query("Who won the most games?")
print(query)

Awesome, let's try running the query and see if we get the expected results.

In [ ]:
def execute_query(query):
    return conn.query(query).fetchdf().to_dict(orient="records")


execute_query(query)

## Evaluation

Evaluation consists of three parts — data, task, and scores. We'll start with data.

In [ ]:
questions = [
    "Which team won the most games?",
    "Which team won the most games in 2015?",
    "Who led the league in 3 point shots?",
    "Which team had the biggest difference in records across two consecutive years?",
    "What is the average number of free throws per year?",
]

Let's store the data above as a versioned dataset in phoenix.

In [ ]:
from uuid import uuid4

import pandas as pd

ds = px.Client().upload_dataset(
    pd.DataFrame([{"question": question} for question in questions]),
    input_keys=["question"],
    output_keys=[],
    dataset_name="nba-questions-" + str(uuid4()),
)

# If you have already uploaded the dataset, you can fetch it using the following line
# ds = px.Client().get_dataset(name="nba-questions")

Next, we'll define the task. The task is to generate SQL queries from natural language questions.

In [ ]:
async def text2sql(question):
    query = await generate_query(question)
    results = None
    error = None
    try:
        results = execute_query(query)
    except duckdb.Error as e:
        error = str(e)

    return {
        "query": query,
        "results": results,
        "error": error,
    }

Finally, we'll define the scores. We'll use the following simple scoring function:

In [ ]:
def no_error(output):
    return 1.0 if output.get("error") is None else 0.0


def has_results(output):
    results = output.get("results")
    print(results)
    has_results = results is not None and len(results) > 0
    return 1.0 if has_results else 0.0

Now let's run the evaluation experiment.

In [ ]:
import phoenix as px
from phoenix.datasets.experiments import run_experiment


# Define the task to run text2sql on the input question
def task(example):
    return text2sql(example.input["question"])


experiment = run_experiment(ds, task=task, evaluators=[no_error, has_results])

Ok! It looks like 3/5 of our queries are valid.


## Interpreting the results

Now that we ran the initial evaluation, it looks like two of the results are valid, two produce SQL errors, and one is incorrect.

- The incorrect query didn't seem to get the date format correct. That would probably be improved by showing a sample of the data to the model (e.g. few shot example).

- There are two binder errors, which may also have to do with not understanding the data format.



Let's try to improve the prompt with few-shot examples and see if we can get better results.

In [ ]:
samples = conn.query("SELECT * FROM nba LIMIT 1").to_df().to_dict(orient="records")[0]
sample_rows = "\n".join(
    f"{column['column_name']} | {column['column_type']} | {samples[column['column_name']]}"
    for column in columns
)
system_prompt = dedent(
    f"""
You are a SQL expert, and you are given a single table named nba with the following columns:

Column | Type | Example
-------|------|--------
{sample_rows}

Write a DuckDB SQL query corresponding to the user's request. Return just the query text, with no formatting (backticks, markdown, etc.).
"""
)


async def generate_query(input):
    response = await client.chat.completions.create(
        model=TASK_MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return response.choices[0].message.content


print(await generate_query("Which team won the most games in 2015?"))

Looking much better! Finally, let's add a scoring function that compares the results, if they exist, with the expected results.




In [ ]:
experiment = run_experiment(ds, task=task, evaluators=[has_results, no_error])

Amazing. It looks like we removed one of the errors, and got a result for the incorrect query. Let's try out using LLM as a judge to see how well it can assess the results.


In [ ]:
from phoenix.datasets.evaluators.llm_evaluators import LLMCriteriaEvaluator
from phoenix.datasets.experiments import evaluate_experiment
from phoenix.evals.models import OpenAIModel

llm_evaluator = LLMCriteriaEvaluator(
    name="is_sql",
    criteria="is_sql",
    description="the output is a valid SQL query and that it executes without errors",
    model=OpenAIModel(),
)

evaluate_experiment(experiment, evaluators=[llm_evaluator])